In [6]:
import sys
sys.path.append('./models/')
from fcunet import fcunet_model
from irnet import irnet_model
from fcmnr import fcmnr_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import math


from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [7]:
bankruptcy_df = pd.read_csv("./data/bankruptcy_dataset.csv")
bankruptcy_df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1,0.035490


In [8]:
X = bankruptcy_df.iloc[:,1:]
Y = bankruptcy_df.iloc[:,[0]]
Y=to_categorical(Y)
scaler = RobustScaler()
X.iloc[:,:] = scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, Y , test_size=0.3, random_state=42, stratify=Y)

/root/anaconda3/envs/gpu/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/root/anaconda3/envs/gpu/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current

In [9]:
model=irnet_model(3,x_train.shape[1],'relu',0.0001)
history = model.fit(x_train,y_train,
                        batch_size=128,
                        epochs=200,
                        validation_split=0.2,
                        callbacks=[EarlyStopping(patience=20)])

prediction=model.predict(x_test)


Train on 3818 samples, validate on 955 samples
Epoch 1/200
3818/3818 [==============================] - 4s 1ms/sample - loss: 0.5110 - accuracy: 0.9508 - val_loss: 0.4431 - val_accuracy: 0.9654
Epoch 2/200
3818/3818 [==============================] - 1s 187us/sample - loss: 0.4688 - accuracy: 0.9597 - val_loss: 0.4244 - val_accuracy: 0.9654
Epoch 3/200
3818/3818 [==============================] - ETA: 0s - loss: 0.4423 - accuracy: 0.95 - 1s 189us/sample - loss: 0.4455 - accuracy: 0.9599 - val_loss: 0.4156 - val_accuracy: 0.9665
Epoch 4/200
3818/3818 [==============================] - 1s 195us/sample - loss: 0.4302 - accuracy: 0.9612 - val_loss: 0.4211 - val_accuracy: 0.9654
Epoch 5/200
3818/3818 [==============================] - 1s 189us/sample - loss: 0.4229 - accuracy: 0.9620 - val_loss: 0.3870 - val_accuracy: 0.9686
Epoch 6/200
3818/3818 [==============================] - 1s 188us/sample - loss: 0.4215 - accuracy: 0.9610 - val_loss: 0.3851 - val_accuracy: 0.9675
Epoch 7/200
3818/38

In [11]:
print(classification_report(np.argmax(y_test, axis=1),np.argmax(prediction, axis=1)))


              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1980
           1       0.13      0.09      0.11        66

    accuracy                           0.95      2046
   macro avg       0.55      0.54      0.54      2046
weighted avg       0.94      0.95      0.95      2046

